# Compress and filter

> Compress the data and save them in pyarrow format, so that easy can be transferred and filter them

In [ ]:
#| default_exp compress_and_filter

In [1]:
#| export 
import os
from PIL import Image
from pathlib import Path
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from typing import Union, List, Tuple, Callable
import numpy as np

from tqdm.auto import tqdm
import os
from fastcore.all import *

In [2]:
#| export
def apply_functions(
        fn: Union[Path, str], # Name fo the file
        functions:Union[List[Callable], None]=None
    ):
    'Apply a list of functions to a file'

    results = {}
    for func in functions:

        try:
            result = func(fn)

            if result is not None:
                results[func.__name__] = result
        except Exception as e:
            print(f'Error in {func.__name__} for {fn}: {e}')
    return results

In [ ]:
#| export
def convert_images_to_parquet(
        image_directory:Union[str, Path],  # directory containing images
        output_file:Union[str, Path],      # output parquet file
        file_name_func:List[Callable]=None, # functions to apply to the filename
        file_exts:str='.png'
        ):
    ' Convert images in a directory to a parquet file '
    images = image_directory.ls(file_exts=file_exts) 
    n_ims = len(images)

    data = []
    for i, im in enumerate(tqdm(images, total=n_ims)):
        fn = im.name
        with Image.open(im) as img:
            img_byte_arr = io.BytesIO()
            img.save(img_byte_arr, format='PNG')
            img_byte_arr = img_byte_arr.getvalue()


            # Apply functions to filename and collect metadata
            if file_name_func is not None:
                metadata = apply_functions(fn, file_name_func)

            data_entry = {'filename': fn, 'image_data': img_byte_arr}
            if file_name_func is not None:
                data_entry.update(metadata)


            # Append the binary data and filename to the list
            data.append(data_entry)

    # Convert the list to a pandas DataFrame
    df = pd.DataFrame(data)
    table = pa.Table.from_pandas(df)
    pq.write_table(table, output_file)

In [ ]:
data_path = os.getenv('DATA_PATH')

In [ ]:
im_src = Path(f'{data_path}/easy_front/Cropped_Images_png')

convert_images_to_parquet(im_src, f'{data_path}/easy_front/Cropped_Images.parquet')

  0%|          | 0/1639 [00:00<?, ?it/s]

In [ ]:
def example_metadata_extractor(filename):
    if "A" in filename:
        return "A_type_pin"
    return "B_type_pin"

In [ ]:
convert_images_to_parquet(im_src, f'{data_path}/easy_front/Cropped_Images.parquet', file_name_func=[example_metadata_extractor])

  0%|          | 0/1639 [00:00<?, ?it/s]

In [ ]:
#| export
def filter_images(parquet_file, condition):
    # Load the Parquet file
    table = pq.read_table(parquet_file)

    # Convert to Pandas DataFrame for easier manipulation
    df = table.to_pandas()

    # Filter the DataFrame based on a condition
    filtered_df = df[df['example_metadata_extractor'].apply(lambda x: condition in x)]

    return filtered_df


In [ ]:
pin_a_images = filter_images(f'{data_path}/easy_front/Cropped_Images.parquet', 'A_type_pin')

In [ ]:
pin_a_images.head()

,filename,image_data,example_metadata_extractor
0,2_1_CroppedImg_20231018_92536201_5_A.png,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,A_type_pin
2,2_1_CroppedImg_20231018_92536201_25_A.png,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,A_type_pin
3,2_2_CroppedImg_20231018_92536201_24_A.png,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,A_type_pin
5,0_1_CroppedImg_20231018_92042317_7_A.png,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,A_type_pin
7,2_2_CroppedImg_20231018_92536201_17_A.png,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,A_type_pin


In [ ]:
#| export
def binary_to_image(data_entry):
    img_data = data_entry['image_data']
    img_byte_arr = io.BytesIO(img_data)
    img = Image.open(img_byte_arr)
    return data_entry['filename'], img

In [ ]:
#| export
def read_images_from_parquet(parquet_path, num_workers=16):
    # Read the Parquet file
    table = pq.read_table(parquet_path)
    df = pd.DataFrame(table.to_pandas())
    
    # Use FastAI's parallel function to process images in parallel
    results = parallel(
        binary_to_image, 
        df.to_dict(orient='records'), 
        n_workers=num_workers,
        progress=True

        )
    
    # Convert list of tuples into a dictionary
    images = {filename: img for filename, img in results}
    return images



In [4]:
#| export
def save_images_to_disk(
        images:Tuple[str, Image.Image],  # Tuple of filename and image
        output_dir:str # where to save the images
        ): 
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    filename, img = images
    img.save(Path(output_dir, filename))

In [5]:
#| export
def decode_batch(
        pr_path:str, # parquet file path
        batch_size:int=1000, # batch size
        num_workers:int=4, # number of workers for parallel processing
        output_dir:str=None, # output directory of saved image
        )->None:
    ' Decode image and in case of output_dir is not None, then save images to path'
    table = pq.read_table(pr_path)
    df = pd.DataFrame(table.to_pandas())
    df_len = len(df)
    split_size = df_len//batch_size
    df_splits = np.array_split(df, split_size)

    def par_save(sn_split,output_dir):

        res = parallel(
            binary_to_image, 
            sn_split.to_dict(orient='records'), 
            n_workers=num_workers,
            progress=False
        )
        
        _ = parallel(
                        save_images_to_disk, 
                        res, 
                        n_workers=num_workers,
                        output_dir=Path(output_dir),
                        progress=False,
                        )


    for i, split_df in tqdm(enumerate(df_splits), total=len(df_splits)):

        results = parallel(
                            binary_to_image, 
                            split_df.to_dict(orient='records'), 
                            n_workers=num_workers,
                            progress=False

        )
        if output_dir is not None:
            _ = parallel(
                        save_images_to_disk, 
                        results, 
                        n_workers=num_workers,
                        output_dir=Path(output_dir),
                        progress=False,
                        threadpool=False
                        )




In [ ]:
#| export
# Define a function to save a single image
def save_image(data):
    filename, image, output_dir = data
    # Ensure the image is in the correct format
    if image.mode not in ['L']:
        image = image.convert('L')
    
    # Create the full path for the image
    file_path = Path(output_dir, filename)
    # Save the image
    image.save(file_path, format='PNG')  # Adjust format as needed


In [8]:
#| hide
import nbdev


In [9]:
#| hide
nbdev.nbdev_export('04_compress_and_filter_data.ipynb')